In [3]:
import numpy as np
import pandas as pd
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [4]:
df = pd.read_csv("../train.csv", index_col=0)
df_independent = df.iloc[:, :-1]


In [15]:
from matplotlib.pyplot import xlabel


def calculate_vif(features: list, data: pd.DataFrame):
    """calculate VIF value for the features given on the data"""
    vif = {}
    for feature in features:
        vif[feature] = float("{:.2f}".format(variance_inflation_factor(data, data.columns.get_loc(feature))))
    return vif


def get_corr(data: pd.DataFrame):
    # Print correlation heatmap
    corr = data.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    cmap = sns.diverging_palette(230, 20, as_cmap=True)

    f, ax = plt.subplots(figsize=(37,37))
    sns.heatmap(corr, mask=mask, cmap=cmap, annot=True)
    
def get_vifs(data: pd.DataFrame):    
    # Get VIF and return it as a series
    data_num = data.select_dtypes(include=["number"])
    vif_feat = calculate_vif(data_num.columns, data_num.dropna(axis=0))
    vif_series = pd.Series(data=vif_feat)
    return vif_series

def get_IQR_outliers(data: pd.DataFrame):
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        IQR = q3 - q1

def get_boxplot(data: pd.Series, label):
    f, ax = plt.subplots(figsize=(25,25))
    ax.boxplot(data, labels=[label])


# EDA
First we will focus in numerical variables. Importantly, 'MSSubClass' is numerical but categorical, as is "OverallQual" and 'OverallCond.' The latter two are at least ordinal (ranked 1-10), so there is a structure to the values. MSSubClass is a simple integer representing a classification, and so should be treated as such. These integers should be converted to strings. 

Let's start by investigating the numerical columns. Looking at the pairwise correlation between different columns, as well as the correlation between columns and the target, SalePrice, can help us eliminate some variables to avoid multicollinearity and reduce the number of variables we need to learn on. Before doing that, we should remove outliers.

In [17]:
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [ ]:
df_num = df.select_dtypes(include=["number"])
for col in df_num.columns:
    get_boxplot(df_num[col], col)

In [19]:
df.describe().to_csv("../data_description.csv")

In [ ]:
get_corr(df_independent)

There do not appear to be any outliers that would be the result of collection errors. Some of the values are very large (for example, a 215,000 sqft lot, or 8,000 sqft home) but these are not unreasonable values (~5 acres of land, and a moderate sized 'mansion' home, repsectively)

Leaving aside the SalePrice for now, there are strong correlations between GarageArea and GarageCars, as well as GarageYrBlt and YearBuilt. TotRmsAbvGr and GrLivArea also show strong correlation, as do 1stFlrSF and TotalBsmtSF. Let's dig into those correlations a little more closely

In [ ]:
print(get_vifs(df_independent))


## YearBuilt vs YearRemodAdd
These two columns have a lot of identical values since YearRemodAdd will just contain YearBuilt if the home was never remodeled. Similarly, many garages are built with the house, so those dates are duplicated. Additional binary columns could be added to indicate that an addition/remodel was done, or a garage was built after initial construction.

## Garage

The Garage size (cars and Area) variables are also closely correlated. We will want to choose one or the other when we get to data wrangling

## Area/Room counts
The variables tracking the area of different rooms and the number of rooms are also highly correlated. This is likely because all floors have similar footprints, so the square footage won't differ much, and more room usually means more SqFt. Given the number of different variables that are correlated, keeping them all introduces lots of multicollinearity. We will want to choose only a subset of these variables.

In [26]:
(df['YearRemodAdd'] - df['YearBuilt']).describe()

count    1460.000000
mean       13.597945
std        24.476465
min         0.000000
25%         0.000000
50%         0.000000
75%        20.000000
max       123.000000
dtype: float64

In [ ]:
area_and_room_vars = ['TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'GrLivArea', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', "SalePrice"]
get_corr(df[area_and_room_vars])

# Electrical

Moving away from numerical variables, I'm interested in the 'Electrical' column. Particularly because I predict it will be highly correlated to the build date.

In [ ]:
df_electrical_year = df.loc[:, ['YearBuilt', "Electrical"]]
with_dummies = pd.get_dummies(df_electrical_year, columns=['Electrical'])
get_corr(with_dummies)

There is a clear correlation between YearBuilt and the home having a circuit breaker. However, circuit breaker is by far the most common type of electrical system, with roughly 1330 samples, compared to the other 4 combined at 130 samples. There is still some logic to retaining all of these variables, since the electrical system being old may significantly lower sale price, due to increased costs to replace electrical equipment if/when it breaks or needs to be replaced in a remodel. 

# Investigate some other variables
With so many variables, its important to see which one's can potentially be discarded, at least at first. I'll check some information from a handful of variables and see if they are mostly identical entries, or if they seem relevant to home price

In [ ]:
lot_features = df.loc[:, ['LandContour', 'LotShape', 'LotConfig', 'SalePrice']]
with_dummies = pd.get_dummies(lot_features)
get_corr(with_dummies)


Some lot features (shape, grade, etc.) do have correlation with sale price and weak correlation with each other, so maybe worth keeping.

In [23]:
df[df['Utilities'] == 'NoSeWa'].to_csv("../no_utils.csv")

All but 1 home, ID 945, had all utilities. That one home seems strange, given that it is in a culdesac. Since 1 entry won't be much help in deciding if utility access is relevant anyway, we can delete the index and then also delete 'Utilities' column.  